In [ ]:
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
#sys.path.append(os.path.join("estadisticas_jupyter.ipynb", '..')) #Preguntar
from rutas import archivo_imagenes_etiquetadas_csv
from rutas import archivo_logs_csv

df=pd.read_csv(archivo_imagenes_etiquetadas_csv)
df_logs=pd.read_csv(archivo_logs_csv)
df

In [ ]:
#inciso 1
df['extension'] = df['mimetype'].str.split('/').str[1]

porcentajes = df['extension'].value_counts()
etiquetas = porcentajes.index

# Generar el gráfico de torta
plt.pie(porcentajes, labels=etiquetas, autopct='%1.1f%%',
        shadow=True, startangle=120, labeldistance=1.1)

plt.axis('equal')
plt.legend(etiquetas)
plt.title("Porcentajes según el tipo de imagen")

plt.show()


In [ ]:
#item 8
df.groupby('ultimo_perfil').sum()['tamaño']
#me tengo que traer los nombres de todos los usuarios y hacer un merge

In [ ]:
#item 7
df2=df.copy()
df2['tags']=df['tags'].str.split(',')
df2['tags'].explode()